In [1]:
from dotenv import dotenv_values
import os

env_vars = dotenv_values('.env')

os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY');

In [7]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Loegnah")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Loegnah
    You:


> Finished chain.


'Hello Loegnah! How can I assist you today?'

In [8]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Loegnah
AI: Hello Loegnah! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


"That's great! Seoul is a vibrant and bustling city. How can I assist you today, Loegnah?"